In [2]:
from elasticsearch import Elasticsearch

# Create an Elasticsearch client instance with authentication
es = Elasticsearch(
    "https://localhost:9200",  # Use https for SSL
    basic_auth=(
        "elastic",
        "8+6x+mXMgyoyi0F7Fi07",
    ),  # Replace with your username and password
    verify_certs=False,  # Optional: Disable SSL certificate verification for development
    ssl_show_warn=False,  # This will disable the warning
)

In [5]:
# Adding orders index with field mappings
response_create_orders_index = es.indices.create(
    index="orders",
    body={
        "mappings": {
            "properties": {
                "purchased_at": {"type": "date"},
                "lines": {
                    "type": "nested",
                    "properties": {
                        "product_id": {"type": "integer"},
                        "amount": {"type": "double"},
                        "quantity": {"type": "short"},
                    },
                },
                "total_amount": {"type": "double"},
                "status": {"type": "keyword"},
                "sales_channel": {"type": "keyword"},
                "salesman": {
                    "type": "object",
                    "properties": {"id": {"type": "integer"}, "name": {"type": "text"}},
                },
            }
        }
    },
    ignore=400,
)

# Output the response to verify index creation
print("Orders index creation response:", response_create_orders_index)

Orders index creation response: {'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [orders/6JA6A5Q0RXmpFrMf-RCOpQ] already exists', 'index_uuid': '6JA6A5Q0RXmpFrMf-RCOpQ', 'index': 'orders'}], 'type': 'resource_already_exists_exception', 'reason': 'index [orders/6JA6A5Q0RXmpFrMf-RCOpQ] already exists', 'index_uuid': '6JA6A5Q0RXmpFrMf-RCOpQ', 'index': 'orders'}, 'status': 400}


/tmp/ipykernel_4694/2907945610.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response_create_orders_index = es.indices.create(


In [5]:
# Calculating statistics with sum, avg, min, and max aggregations
response_sales_statistics = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "total_sales": {"sum": {"field": "total_amount"}},
            "avg_sale": {"avg": {"field": "total_amount"}},
            "min_sale": {"min": {"field": "total_amount"}},
            "max_sale": {"max": {"field": "total_amount"}},
        },
    },
)

# Retrieving the number of distinct values
response_distinct_salesmen = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {"total_salesmen": {"cardinality": {"field": "salesman.id"}}},
    },
)

# Retrieving the number of values
response_values_count = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {"values_count": {"value_count": {"field": "total_amount"}}},
    },
)

# Using stats aggregation for common statistics
response_amount_stats = es.search(
    index="orders",
    body={"size": 0, "aggs": {"amount_stats": {"stats": {"field": "total_amount"}}}},
)

# Output the responses to verify the results
print("Sales Statistics:", response_sales_statistics)
print("Distinct Salesmen Count:", response_distinct_salesmen)
print("Total Values Count:", response_values_count)
print("Amount Stats:", response_amount_stats)

Sales Statistics: {'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'total_sales': {'value': 109209.61}, 'avg_sale': {'value': 109.20961}, 'min_sale': {'value': 10.27}, 'max_sale': {'value': 281.77}}}
Distinct Salesmen Count: {'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'total_salesmen': {'value': 100}}}
Total Values Count: {'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'values_count': {'value': 1000}}}
Amount Stats: {'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'t

bucket aggregations

In [6]:
# Creating a bucket for each status value
response_status_terms = es.search(
    index="orders",
    body={"size": 0, "aggs": {"status_terms": {"terms": {"field": "status"}}}},
)

# Including 20 terms instead of the default 10
response_status_terms_size = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {"status_terms": {"terms": {"field": "status", "size": 20}}},
    },
)

# Aggregating documents with missing field (or NULL)
response_status_terms_missing = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "status_terms": {"terms": {"field": "status", "size": 20, "missing": "N/A"}}
        },
    },
)

# Changing the minimum document count for a bucket to be created
response_status_terms_min_doc_count = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "status_terms": {
                "terms": {
                    "field": "status",
                    "size": 20,
                    "missing": "N/A",
                    "min_doc_count": 0,
                }
            }
        },
    },
)

# Ordering the buckets
response_status_terms_ordered = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "status_terms": {
                "terms": {
                    "field": "status",
                    "size": 20,
                    "missing": "N/A",
                    "min_doc_count": 0,
                    "order": {"_key": "asc"},
                }
            }
        },
    },
)

# Output the responses to verify the results
print("Status Terms Aggregation:", response_status_terms)
print("Status Terms with Size 20:", response_status_terms_size)
print("Status Terms with Missing Field:", response_status_terms_missing)
print("Status Terms with Min Doc Count:", response_status_terms_min_doc_count)
print("Ordered Status Terms:", response_status_terms_ordered)

Status Terms Aggregation: {'took': 192, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'status_terms': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'processed', 'doc_count': 209}, {'key': 'completed', 'doc_count': 204}, {'key': 'pending', 'doc_count': 199}, {'key': 'cancelled', 'doc_count': 196}, {'key': 'confirmed', 'doc_count': 192}]}}}
Status Terms with Size 20: {'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'status_terms': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'processed', 'doc_count': 209}, {'key': 'completed', 'doc_count': 204}, {'key': 'pending', 'doc_count': 199}, {'key': 'cancelled', 'doc_count': 196}, {'

nested aggregations

In [7]:
# Retrieving statistics for each status
response_status_statistics = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "status_terms": {
                "terms": {"field": "status"},
                "aggs": {"status_stats": {"stats": {"field": "total_amount"}}},
            }
        },
    },
)

# Narrowing down the aggregation context
response_narrowed_aggregation = es.search(
    index="orders",
    body={
        "size": 0,
        "query": {"range": {"total_amount": {"gte": 100}}},
        "aggs": {
            "status_terms": {
                "terms": {"field": "status"},
                "aggs": {"status_stats": {"stats": {"field": "total_amount"}}},
            }
        },
    },
)

# Output the responses to verify the results
print("Status Statistics Aggregation:", response_status_statistics)
print("Narrowed Down Aggregation:", response_narrowed_aggregation)

Status Statistics Aggregation: {'took': 49, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'status_terms': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'processed', 'doc_count': 209, 'status_stats': {'count': 209, 'min': 10.27, 'max': 281.77, 'avg': 109.30703349282295, 'sum': 22845.17}}, {'key': 'completed', 'doc_count': 204, 'status_stats': {'count': 204, 'min': 10.93, 'max': 260.59, 'avg': 113.54058823529411, 'sum': 23162.28}}, {'key': 'pending', 'doc_count': 199, 'status_stats': {'count': 199, 'min': 10.8, 'max': 260.03, 'avg': 115.66125628140703, 'sum': 23016.59}}, {'key': 'cancelled', 'doc_count': 196, 'status_stats': {'count': 196, 'min': 10.7, 'max': 272.9, 'avg': 107.19158163265305, 'sum': 21009.55}}, {'key': 'confirmed', 'doc_count': 192, 'status_stats': {'count': 192, 'min': 11.01, 'max': 246.89, 'a

In [8]:
# Filtering out documents with low total_amount
response_low_value = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {"low_value": {"filter": {"range": {"total_amount": {"lt": 50}}}}},
    },
)

# Aggregating on the bucket of remaining documents
response_low_value_avg = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "low_value": {
                "filter": {"range": {"total_amount": {"lt": 50}}},
                "aggs": {"avg_amount": {"avg": {"field": "total_amount"}}},
            }
        },
    },
)

# Output the responses to verify the results
print("Low Value Filter Aggregation:", response_low_value)
print("Low Value Average Amount Aggregation:", response_low_value_avg)

Low Value Filter Aggregation: {'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'low_value': {'doc_count': 164}}}
Low Value Average Amount Aggregation: {'took': 12, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'low_value': {'doc_count': 164, 'avg_amount': {'value': 32.59371951219512}}}}


bucket rules for filters

In [9]:
# Placing documents into buckets based on criteria
response_buckets = es.search(
    index="recipes",
    body={
        "size": 0,
        "aggs": {
            "my_filter": {
                "filters": {
                    "filters": {
                        "pasta": {"match": {"title": "pasta"}},
                        "spaghetti": {"match": {"title": "spaghetti"}},
                    }
                }
            }
        },
    },
)

# Calculate average ratings for buckets
response_avg_ratings = es.search(
    index="recipes",
    body={
        "size": 0,
        "aggs": {
            "my_filter": {
                "filters": {
                    "filters": {
                        "pasta": {"match": {"title": "pasta"}},
                        "spaghetti": {"match": {"title": "spaghetti"}},
                    }
                },
                "aggs": {"avg_rating": {"avg": {"field": "ratings"}}},
            }
        },
    },
)

# Output the responses to verify the results
print("Buckets Response:", response_buckets)
print("Average Ratings Response:", response_avg_ratings)

Buckets Response: {'took': 21, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'my_filter': {'buckets': {'pasta': {'doc_count': 0}, 'spaghetti': {'doc_count': 0}}}}}
Average Ratings Response: {'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'my_filter': {'buckets': {'pasta': {'doc_count': 0, 'avg_rating': {'value': None}}, 'spaghetti': {'doc_count': 0, 'avg_rating': {'value': None}}}}}}


range aggregations

In [10]:
# Range aggregation for total amount
response_amount_distribution = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "amount_distribution": {
                "range": {
                    "field": "total_amount",
                    "ranges": [{"to": 50}, {"from": 50, "to": 100}, {"from": 100}],
                }
            }
        },
    },
)

# Date range aggregation
response_purchased_ranges = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "purchased_ranges": {
                "date_range": {
                    "field": "purchased_at",
                    "ranges": [
                        {"from": "2016-01-01", "to": "2016-01-01||+6M"},
                        {"from": "2016-01-01||+6M", "to": "2016-01-01||+1y"},
                    ],
                }
            }
        },
    },
)

# Specifying the date format
response_purchased_ranges_with_format = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "purchased_ranges": {
                "date_range": {
                    "field": "purchased_at",
                    "format": "yyyy-MM-dd",
                    "ranges": [
                        {"from": "2016-01-01", "to": "2016-01-01||+6M"},
                        {"from": "2016-01-01||+6M", "to": "2016-01-01||+1y"},
                    ],
                }
            }
        },
    },
)

# Enabling keys for the buckets
response_purchased_ranges_keyed = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "purchased_ranges": {
                "date_range": {
                    "field": "purchased_at",
                    "format": "yyyy-MM-dd",
                    "keyed": True,
                    "ranges": [
                        {"from": "2016-01-01", "to": "2016-01-01||+6M"},
                        {"from": "2016-01-01||+6M", "to": "2016-01-01||+1y"},
                    ],
                }
            }
        },
    },
)

# Defining the bucket keys
response_purchased_ranges_with_keys = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "purchased_ranges": {
                "date_range": {
                    "field": "purchased_at",
                    "format": "yyyy-MM-dd",
                    "keyed": True,
                    "ranges": [
                        {
                            "from": "2016-01-01",
                            "to": "2016-01-01||+6M",
                            "key": "first_half",
                        },
                        {
                            "from": "2016-01-01||+6M",
                            "to": "2016-01-01||+1y",
                            "key": "second_half",
                        },
                    ],
                }
            }
        },
    },
)

# Adding a sub-aggregation for bucket stats
response_purchased_ranges_with_subagg = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "purchased_ranges": {
                "date_range": {
                    "field": "purchased_at",
                    "format": "yyyy-MM-dd",
                    "keyed": True,
                    "ranges": [
                        {
                            "from": "2016-01-01",
                            "to": "2016-01-01||+6M",
                            "key": "first_half",
                        },
                        {
                            "from": "2016-01-01||+6M",
                            "to": "2016-01-01||+1y",
                            "key": "second_half",
                        },
                    ],
                },
                "aggs": {"bucket_stats": {"stats": {"field": "total_amount"}}},
            }
        },
    },
)

# Output the responses to verify the results
print("Amount Distribution Response:", response_amount_distribution)
print("Purchased Ranges Response:", response_purchased_ranges)
print("Purchased Ranges with Format Response:", response_purchased_ranges_with_format)
print("Purchased Ranges Keyed Response:", response_purchased_ranges_keyed)
print("Purchased Ranges with Keys Response:", response_purchased_ranges_with_keys)
print(
    "Purchased Ranges with Sub-aggregation Response:",
    response_purchased_ranges_with_subagg,
)

Amount Distribution Response: {'took': 29, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'amount_distribution': {'buckets': [{'key': '*-50.0', 'to': 50.0, 'doc_count': 164}, {'key': '50.0-100.0', 'from': 50.0, 'to': 100.0, 'doc_count': 347}, {'key': '100.0-*', 'from': 100.0, 'doc_count': 489}]}}}
Purchased Ranges Response: {'took': 48, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'purchased_ranges': {'buckets': [{'key': '2016-01-01T00:00:00.000Z-2016-07-01T00:00:00.000Z', 'from': 1451606400000.0, 'from_as_string': '2016-01-01T00:00:00.000Z', 'to': 1467331200000.0, 'to_as_string': '2016-07-01T00:00:00.000Z', 'doc_count': 481}, {'key': '2016-07-01T00:00:00.000Z-2017-01-01T00:00:00.000Z', 'from': 146733

histograms

In [11]:
# 1. Distribution of total_amount with interval 25
response_amount_distribution = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "amount_distribution": {
                "histogram": {"field": "total_amount", "interval": 25}
            }
        },
    },
)

# 2. Requiring minimum 1 document per bucket
response_min_doc_count = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "amount_distribution": {
                "histogram": {
                    "field": "total_amount",
                    "interval": 25,
                    "min_doc_count": 1,
                }
            }
        },
    },
)

# 3. Specifying fixed bucket boundaries
response_fixed_boundaries = es.search(
    index="orders",
    body={
        "size": 0,
        "query": {"range": {"total_amount": {"gte": 100}}},
        "aggs": {
            "amount_distribution": {
                "histogram": {
                    "field": "total_amount",
                    "interval": 25,
                    "min_doc_count": 0,
                    "extended_bounds": {"min": 0, "max": 500},
                }
            }
        },
    },
)

# 4. Aggregating by month with the date_histogram aggregation
response_orders_over_time = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "orders_over_time": {
                "date_histogram": {
                    "field": "purchased_at",
                    "calendar_interval": "month",
                }
            }
        },
    },
)

# Output the responses to verify the results
print("Amount Distribution Response:", response_amount_distribution)
print("Minimum Document Count Response:", response_min_doc_count)
print("Fixed Boundaries Response:", response_fixed_boundaries)
print("Orders Over Time Response:", response_orders_over_time)

Amount Distribution Response: {'took': 13, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'amount_distribution': {'buckets': [{'key': 0.0, 'doc_count': 42}, {'key': 25.0, 'doc_count': 122}, {'key': 50.0, 'doc_count': 153}, {'key': 75.0, 'doc_count': 194}, {'key': 100.0, 'doc_count': 124}, {'key': 125.0, 'doc_count': 125}, {'key': 150.0, 'doc_count': 89}, {'key': 175.0, 'doc_count': 71}, {'key': 200.0, 'doc_count': 42}, {'key': 225.0, 'doc_count': 33}, {'key': 250.0, 'doc_count': 4}, {'key': 275.0, 'doc_count': 1}]}}}
Minimum Document Count Response: {'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1000, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'amount_distribution': {'buckets': [{'key': 0.0, 'doc_count': 42}, {'key': 25.0, 'doc_count': 122

global aggs

In [12]:
# 1. Break out of the aggregation context
response_break_aggregation = es.search(
    index="orders",
    body={
        "query": {"range": {"total_amount": {"gte": 100}}},
        "size": 0,
        "aggs": {
            "all_orders": {
                "global": {},
                "aggs": {"stats_amount": {"stats": {"field": "total_amount"}}},
            }
        },
    },
)

# 2. Adding aggregation without global context
response_additional_aggregation = es.search(
    index="orders",
    body={
        "query": {"range": {"total_amount": {"gte": 100}}},
        "size": 0,
        "aggs": {
            "all_orders": {
                "global": {},
                "aggs": {"stats_amount": {"stats": {"field": "total_amount"}}},
            },
            "stats_expensive": {"stats": {"field": "total_amount"}},
        },
    },
)

# Output the responses to verify the results
print("Break Aggregation Context Response:", response_break_aggregation)
print(
    "Additional Aggregation without Global Context Response:",
    response_additional_aggregation,
)

Break Aggregation Context Response: {'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 489, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'all_orders': {'doc_count': 1000, 'stats_amount': {'count': 1000, 'min': 10.27, 'max': 281.77, 'avg': 109.20961, 'sum': 109209.61}}}}
Additional Aggregation without Global Context Response: {'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 489, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'all_orders': {'doc_count': 1000, 'stats_amount': {'count': 1000, 'min': 10.27, 'max': 281.77, 'avg': 109.20961, 'sum': 109209.61}}, 'stats_expensive': {'count': 489, 'min': 100.05, 'max': 281.77, 'avg': 157.32703476482618, 'sum': 76932.92}}}


missing values or fiels

In [13]:
# Adding test documents
# Document 1001
es.index(index="orders", id="1001", body={"total_amount": 100})
# Document 1002 with a missing 'status' field
es.index(index="orders", id="1002", body={"total_amount": 200, "status": None})

# Refresh the index to make the documents searchable
es.indices.refresh(index="orders")

# Aggregating documents with missing field value
response_missing_field = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {"orders_without_status": {"missing": {"field": "status"}}},
    },
)

# Combining missing aggregation with other aggregations
response_combined_aggregation = es.search(
    index="orders",
    body={
        "size": 0,
        "aggs": {
            "orders_without_status": {
                "missing": {"field": "status"},
                "aggs": {"missing_sum": {"sum": {"field": "total_amount"}}},
            }
        },
    },
)

# Deleting test documents
es.delete(index="orders", id="1001")
es.delete(index="orders", id="1002")

# Output the responses to verify the results
print("Missing Field Aggregation Response:", response_missing_field)
print("Combined Aggregation Response:", response_combined_aggregation)

Missing Field Aggregation Response: {'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1002, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'orders_without_status': {'doc_count': 2}}}
Combined Aggregation Response: {'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1002, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'orders_without_status': {'doc_count': 2, 'missing_sum': {'value': 300.0}}}}


aggregations on nested values

In [15]:
# Adding sample data to the 'department' index
es.index(
    index="department",
    id="1",
    body={
        "name": "Sales",
        "employees": [
            {"name": "Alice", "age": 30},
            {"name": "Bob", "age": 25},
            {"name": "Charlie", "age": 35},
        ],
    },
)

es.index(
    index="department",
    id="2",
    body={
        "name": "Marketing",
        "employees": [{"name": "David", "age": 40}, {"name": "Eve", "age": 28}],
    },
)

# Refresh the index to make the documents searchable
es.indices.refresh(index="department")

# Aggregating nested objects (counting employees)
response_nested_aggregation = es.search(
    index="department",
    body={"size": 0, "aggs": {"employees": {"nested": {"path": "employees"}}}},
)

# Aggregating to find the minimum age of employees
response_min_age_aggregation = es.search(
    index="department",
    body={
        "size": 0,
        "aggs": {
            "employees": {
                "nested": {"path": "employees"},
                "aggs": {"minimum_age": {"min": {"field": "employees.age"}}},
            }
        },
    },
)

# Deleting sample documents for cleanup
es.delete(index="department", id="1")
es.delete(index="department", id="2")

# Output the responses to verify the results
print("Nested Aggregation Response:", response_nested_aggregation)
print("Minimum Age Aggregation Response:", response_min_age_aggregation)

Nested Aggregation Response: {'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'employees': {'doc_count': 5}}}
Minimum Age Aggregation Response: {'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'employees': {'doc_count': 5, 'minimum_age': {'value': 25.0}}}}
